### Targeted information retrieval

We have seen how to parse a page, retrieve without distinction the information of a web page

But, in general, the purpose of scrapping is to automate the collection of targeted information on the web


In [2]:
from bs4 import BeautifulSoup
import requests


Let's say I want to scrape the description of the latest movies released in theaters

So I go to the allo ciné site and try to find the tags that will give me links to the specific pages of these films to get their summaries.

#### Recovery of the url of the pages of films freshly released in the cinema

In [4]:
url='http://www.allocine.fr/'
r = requests.get(url)
print(url, r.status_code)
soup = BeautifulSoup(r.content,'lxml')
soup_list = str(soup).split('\n')
for i in soup_list[:10]:
    print(i)

http://www.allocine.fr/ 200
<!DOCTYPE html>
<html lang="fr">
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="index,follow,max-snippet:-1" name="robots"/>
<title>AlloCiné : Cinéma, Séries TV, BO de films et séries, Vidéos, DVD et VOD</title>
<meta content="" name="keywords"/>
<meta content="noarchive" name="Googlebot"/>


On my web browser (Chrome, mozilla...) I use the "inspect" function (right click / inspect) and I drag my mouse to the areas of the page that interest me. At the same time I will see the html script move to the instructions of the html script in question. 

That's how I do it to find the tags I'm interested in.

I notice that the relative link of the web page specific to the new movie on the poster is stored in these tags:

a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=235582.html" title="Le Grand Bain">Le Grand Bain</a


In [7]:
for p in soup.find_all('a')[:7]:
    print (p.text)

Top Gun 2: Maverick
The Last Duel
Prodigal Son
Modern Family

            Cinéma
        

            Séries
        

            Trailers
        


This time the site is more difficult to "extract", let's use additional parameters to this search function, find_all, much more specific than the one currently used

In [8]:
# In addition to the tag a, well identified, we notice some additional 
# information such as the value of the class variable of these identical tags
for elem in soup.find_all('a',attrs={"class" :"meta-title meta-title-link"}):
    print(elem)

<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=272217.html" title="Antoinette dans les Cévennes">Antoinette dans les Cévennes</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=274282.html" title="J’irai mourir dans les Carpates">J’irai mourir dans les Carpates</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=278943.html" title="Les Choses qu’on dit, les choses qu’on fait">Les Choses qu’on dit, les choses qu’on fait</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=281582.html" title="Une nuit au Louvre: Léonard de Vinci">Une nuit au Louvre: Léonard de Vinci</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=283952.html" title="Les Mal-aimés">Les Mal-aimés</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=270321.html" title="Honeyland">Honeyland</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=281998.html" title="Africa M

#### Recovery of href

We have noticed the presence of href links to the pages that interest us. Let's go get them back:

In [9]:
for elem in soup.find_all('a',attrs={"class" :"meta-title meta-title-link"}):
    print(elem.get('href'))
    # return a list

/film/fichefilm_gen_cfilm=272217.html
/film/fichefilm_gen_cfilm=274282.html
/film/fichefilm_gen_cfilm=278943.html
/film/fichefilm_gen_cfilm=281582.html
/film/fichefilm_gen_cfilm=283952.html
/film/fichefilm_gen_cfilm=270321.html
/film/fichefilm_gen_cfilm=281998.html
/film/fichefilm_gen_cfilm=277759.html
/film/fichefilm_gen_cfilm=279600.html
/film/fichefilm_gen_cfilm=285874.html
/film/fichefilm_gen_cfilm=251315.html
/film/fichefilm_gen_cfilm=275608.html
/film/fichefilm_gen_cfilm=269704.html
/film/fichefilm_gen_cfilm=267302.html
/film/fichefilm_gen_cfilm=268644.html
/film/fichefilm_gen_cfilm=271602.html
/film/fichefilm_gen_cfilm=237864.html
/film/fichefilm_gen_cfilm=258205.html
/series/ficheserie_gen_cserie=26001.html
/series/ficheserie_gen_cserie=24218.html
/series/ficheserie_gen_cserie=25147.html
/series/ficheserie_gen_cserie=25236.html
/series/ficheserie_gen_cserie=25046.html
/series/ficheserie_gen_cserie=25304.html


Can you retrieve the titles for me via the search for "title" in the items of the previous list?

In [10]:
for elem in soup.find_all('a', attrs={"class":"meta-title meta-title-link"}):
    print(elem.get('title'))

Antoinette dans les Cévennes
J’irai mourir dans les Carpates
Les Choses qu’on dit, les choses qu’on fait
Une nuit au Louvre: Léonard de Vinci
Les Mal-aimés
Honeyland
Africa Mia
Trio
La Maquisarde
Mathilde et Rosette
Tenet
Effacer l’historique
Petit Pays
Police
La Daronne
Le Bonheur des uns...
Les Nouveaux mutants
Enorme
Grand Hôtel (2020)
Family Business
La Révolution
No Man's Land
The Walking Dead: World Beyond
The Haunting of Bly Manor


#### Get summary

Let's start by building the url that we will use to retrieve the summaries

Start by putting the href values in a list of links


In [11]:
links=[]
for elem in soup.find_all('a',attrs={"class" :"meta-title meta-title-link"}):
    # Tout simplement
    links.append(elem.get('href'))
    # Je range tout cela dans une liste 
links

['/film/fichefilm_gen_cfilm=272217.html',
 '/film/fichefilm_gen_cfilm=274282.html',
 '/film/fichefilm_gen_cfilm=278943.html',
 '/film/fichefilm_gen_cfilm=281582.html',
 '/film/fichefilm_gen_cfilm=283952.html',
 '/film/fichefilm_gen_cfilm=270321.html',
 '/film/fichefilm_gen_cfilm=281998.html',
 '/film/fichefilm_gen_cfilm=277759.html',
 '/film/fichefilm_gen_cfilm=279600.html',
 '/film/fichefilm_gen_cfilm=285874.html',
 '/film/fichefilm_gen_cfilm=251315.html',
 '/film/fichefilm_gen_cfilm=275608.html',
 '/film/fichefilm_gen_cfilm=269704.html',
 '/film/fichefilm_gen_cfilm=267302.html',
 '/film/fichefilm_gen_cfilm=268644.html',
 '/film/fichefilm_gen_cfilm=271602.html',
 '/film/fichefilm_gen_cfilm=237864.html',
 '/film/fichefilm_gen_cfilm=258205.html',
 '/series/ficheserie_gen_cserie=26001.html',
 '/series/ficheserie_gen_cserie=24218.html',
 '/series/ficheserie_gen_cserie=25147.html',
 '/series/ficheserie_gen_cserie=25236.html',
 '/series/ficheserie_gen_cserie=25046.html',
 '/series/ficheseri

The absolute url of the searched film pages is built in this form: http://www.allocine.fr/film/fichefilm_gen_cfilm=243835.html

It is therefore necessary to repeat the previous list and build the absolute urls for our search

It's up to you to play

nb: Do not take the links for the series

In [12]:
links_movie=['http://www.allocine.fr'+ elem for elem in links if 'film' in elem]
links_movie

['http://www.allocine.fr/film/fichefilm_gen_cfilm=272217.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=274282.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=278943.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=281582.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=283952.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=270321.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=281998.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=277759.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=279600.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=285874.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=251315.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=275608.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=269704.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=267302.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=268644.html',
 'http://www.allocine.fr/film/fichefilm_

Finally, on each page, the title and synopsis must be retrieved. Let's try for a movie, the first of the link list

In [13]:
for url in links_movie[:3]:
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    title = soup.find_all('div', {'class':'titlebar-title titlebar-title-lg'})
    print("\u0332".join(title[0].text))
    synopsis = soup.find_all('div', {'class':'content-txt'})
    print(synopsis[0].text.strip())
    print()

A̲n̲t̲o̲i̲n̲e̲t̲t̲e̲ ̲d̲a̲n̲s̲ ̲l̲e̲s̲ ̲C̲é̲v̲e̲n̲n̲e̲s
Le film fait partie de la Sélection Officielle Cannes 2020Des mois qu’Antoinette attend l’été et la promesse d’une semaine en amoureux avec son amant, Vladimir. Alors quand celui-ci annule leurs vacances pour partir marcher dans les Cévennes avec sa femme et sa fille, Antoinette ne réfléchit pas longtemps : elle part sur ses traces ! Mais à son arrivée, point de Vladimir - seulement Patrick, un âne récalcitrant qui va l'accompagner dans son singulier périple…

J̲’̲i̲r̲a̲i̲ ̲m̲o̲u̲r̲i̲r̲ ̲d̲a̲n̲s̲ ̲l̲e̲s̲ ̲C̲a̲r̲p̲a̲t̲e̲s
L’histoire commence par un banal accident de voiture sur une route montagneuse des Carpates. La voiture d'Antoine de Maximy, le présentateur de la série "J'irai dormir chez vous" a été emportée dans une rivière et son corps n’a pas été retrouvé. Le matériel et les images du globe-squatteur sont rapatriés à Paris. Agnès, la monteuse de la série, décide de terminer ce dernier épisode. Après avoir visionné les images

For title 

div class="titlebar-title titlebar-title-lg">Le Grand Bain</div

For the synopsis

div class="content-txt " itemprop="description"
    
              C’est dans les couloirs de leur piscine municipale que Bertrand, Marcus, Simon, Laurent, Thierry et les autres s’entraînent sous l’autorité toute relative de Delphine, ancienne gloire des bassins. Ensemble, ils se sentent libres et utiles. Ils vont mettre toute leur énergie dans une discipline jusque-là propriété de la gent féminine : la natation synchronisée. Alors, oui c’est une idée plutôt bizarre, mais ce défi leur permettra de trouver un sens à leur vie...
    
      </div

1) Automate this script for the entire list

2) Put the information in three lists (film_links, title, synopsis)

3) Create a dataframe that includes these three informations in three associated columns

4) Save this dataframe in a csv file

And here's your first real scrap, you're real hackers now.

In [14]:
import time
import random
from random import randint

title=[]
synopsis=[]

for link in links_movie:
    
    url=link
    # I slow down the frequency of requests to avoid being identified and therefore ban from the site
    time.sleep(random.uniform(1.0, 2.0))
    r = requests.get(url)
    print(url, r.status_code)
    soup = BeautifulSoup(r.content,'lxml')
    
    
    for elem in soup.find_all('div',attrs={"class" :"titlebar-title titlebar-title-lg"}):
        title.append(elem.text.strip())

    synopsis.append((soup.find_all('div',attrs={"class" :"content-txt"}))[0].text.strip())

        
# I check the length of the lists before creating the df
len(title),len(synopsis),len(links_movie)

http://www.allocine.fr/film/fichefilm_gen_cfilm=272217.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=274282.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=278943.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=281582.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=283952.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=270321.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=281998.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=277759.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=279600.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=285874.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=251315.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=275608.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=269704.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=267302.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=268644.html 200
http://www.allocine.fr/film/fichefilm_ge

(18, 18, 18)

In [15]:
import pandas as pd
df=pd.DataFrame({'Title':title})
df['synopsis']=synopsis
df['liens']=links_movie

In [16]:
df.to_csv('../../04.File-handling/data/allo_cine.csv', index=False)